<a href="https://colab.research.google.com/github/thisisanu/online_deep_learning/blob/main/homework3/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available()
!nvidia-smi


Mon Nov 10 00:11:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import os
import shutil

# Ensure we are in /content
os.chdir("/content")
print("Current directory:", os.getcwd())

# If a folder named homework2 exists, remove it completely
if os.path.exists("online_deep_learning"):
    shutil.rmtree("online_deep_learning")
    print("Old online_deep_learning folder removed.")
else:
    print("No existing online_deep_learning folder found.")

!git clone https://github.com/thisisanu/online_deep_learning.git
%cd online_deep_learning/homework3


Current directory: /content
Old online_deep_learning folder removed.
Cloning into 'online_deep_learning'...
remote: Enumerating objects: 501, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 501 (delta 137), reused 27 (delta 6), pack-reused 295 (from 2)
Receiving objects: 100% (501/501), 3.06 MiB | 21.76 MiB/s, done.
Resolving deltas: 100% (324/324), done.
/content/online_deep_learning/homework3


In [5]:
!pip install numpy torch torchvision tqdm matplotlib

#download the datasets by running the following command from the main directory
!curl -s -L https://www.cs.utexas.edu/~bzhou/dl_class/classification_data.zip -o ./classification_data.zip && unzip -qo classification_data.zip
!curl -s -L https://www.cs.utexas.edu/~bzhou/dl_class/drive_data.zip -o ./drive_data.zip && unzip -qo drive_data.zip
!ls


bundle.py		 drive_data	 homework	  requirements.txt
classification_data	 drive_data.zip  Homework3.ipynb
classification_data.zip  grader		 README.md


In [7]:
!ls
!python3 -m grader homework -v

bundle.py		 drive_data	 homework	  requirements.txt
classification_data	 drive_data.zip  Homework3.ipynb
classification_data.zip  grader		 README.md
Public grader loaded.
[INFO     00:00:001] Classifier
[INFO     00:00:501]   - Predict                                            [ 10 / 10 ]
[WARNING  00:00:531]   - Accuracy                                           [ 0 / 25 ]
[WARNING  00:00:531] classifier.th not found
[WARNING  00:00:555]   - Accuracy: Extra Credit                             [ 0 / 2 ]
[WARNING  00:00:555] classifier.th not found
[INFO     00:00:556]  --------------------------------------------------    [  10 /  35 ]
[INFO     00:00:557] Detector
[INFO     00:00:603]   - Predict                                            [ 10 / 10 ]
[WARNING  00:00:604]   - Segmentation Accuracy                              [ 0 / 10 ]
[WARNING  00:00:604] detector.th not found
[WARNING  00:00:604]   - Segmentation IoU                                   [ 0 / 25 ]
[WARNING  00:00:604] 

# Task
Implement and train a classification model, saving the trained model as `classifier.th`. Subsequently, implement and train an object detection model, saving the trained model as `detector.th`. Conclude by confirming the successful generation of both `classifier.th` and `detector.th` files.

## Implement Classification Training Setup

### Subtask:
Create the `train_classification.py` script and set up initial components like imports, device configuration, and data loading for the classification task. This will prepare the environment for model definition and training.


**Reasoning**:
The subtask requires creating a new Python file `train_classification.py` and adding initial setup code for imports, device configuration, data transformations, and data loading. This code block will create the file and include all the necessary components as specified in the instructions.



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
import argparse

# 1. Device configuration
def setup_device():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    return device

# 2. Data transformations
def get_data_transforms(image_size):
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms

# 3. Data loading
def get_data_loaders(data_dir, data_transforms, batch_size):
    image_datasets = {
        x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
        for x in ['train', 'val']
    }
    dataloaders = {
        x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2)
        for x in ['train', 'val']
    }
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    class_names = image_datasets['train'].classes
    print(f"Found {dataset_sizes['train']} training images and {dataset_sizes['val']} validation images.")
    print(f"Classes: {class_names}")
    return dataloaders, dataset_sizes, class_names

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Classification Training Setup')
    parser.add_argument('--data_dir', type=str, default='./classification_data', help='Directory with classification data')
    parser.add_argument('--batch_size', type=int, default=32, help='Batch size for data loaders')
    parser.add_argument('--image_size', type=int, default=224, help='Image size for resizing')
    args = parser.parse_args()

    # Create train_classification.py and write the setup code to it
    file_content = f'''
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
import argparse

# Device configuration
def setup_device():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {{device}}")
    return device

# Data transformations
def get_data_transforms(image_size):
    data_transforms = {{
        'train': transforms.Compose([
            transforms.Resize(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }}
    return data_transforms

# Data loading
def get_data_loaders(data_dir, data_transforms, batch_size):
    image_datasets = {{
        x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
        for x in ['train', 'val']
    }}
    dataloaders = {{
        x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2)
        for x in ['train', 'val']
    }}
    dataset_sizes = {{x: len(image_datasets[x]) for x in ['train', 'val']}}
    class_names = image_datasets['train'].classes
    print(f"Found {{dataset_sizes['train']}} training images and {{dataset_sizes['val']}} validation images.")
    print(f"Classes: {{class_names}}")
    return dataloaders, dataset_sizes, class_names

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Classification Training Setup')
    parser.add_argument('--data_dir', type=str, default='./classification_data', help='Directory with classification data')
    parser.add_argument('--batch_size', type=int, default=32, help='Batch size for data loaders')
    parser.add_argument('--image_size', type=int, default=224, help='Image size for resizing')
    args = parser.parse_args()

    device = setup_device()
    data_transforms = get_data_transforms(args.image_size)
    dataloaders, dataset_sizes, class_names = get_data_loaders(args.data_dir, data_transforms, args.batch_size)

    # Example of how to access data
    # inputs, labels = next(iter(dataloaders['train']))
    # print(f"Batch input shape: {{inputs.shape}}")
    # print(f"Batch labels shape: {{labels.shape}}")

    print("Classification training setup complete.")
    '''

    with open('train_classification.py', 'w') as f:
        f.write(file_content)
    print("Created train_classification.py with initial setup code.")

    # Optionally, run the script to test the setup (not part of the file content itself)
    # print("\nRunning train_classification.py to verify setup...")
    # os.system('python train_classification.py')

    print("Initial classification training setup components are ready in train_classification.py.")

usage: colab_kernel_launcher.py [-h] [--data_dir DATA_DIR]
                                [--batch_size BATCH_SIZE]
                                [--image_size IMAGE_SIZE]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-16a2b55d-7aa3-4641-aad2-09b2ce68c51b.json


SystemExit: 2

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**Reasoning**:
The previous code failed because the `argparse.ArgumentParser().parse_args()` method was being called directly within the Jupyter cell, which attempts to parse arguments passed by the Jupyter kernel (like `-f`), leading to an `unrecognized arguments` error. The fix is to remove the `if __name__ == '__main__':` block and its associated `argparse` parsing from the current notebook cell, ensuring that the code inside this block (including `argparse`) is only part of the string written to `train_classification.py` and not executed by the cell itself.



In [2]:
import os

# The content that will be written to train_classification.py
# This string contains all the necessary imports, function definitions,
# and the main execution block for the classification training setup.
file_content = """
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
import argparse

# Device configuration
def setup_device():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    return device

# Data transformations
def get_data_transforms(image_size):
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms

# Data loading
def get_data_loaders(data_dir, data_transforms, batch_size):
    image_datasets = {
        x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
        for x in ['train', 'val']
    }
    dataloaders = {
        x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2)
        for x in ['train', 'val']
    }
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    class_names = image_datasets['train'].classes
    print(f"Found {dataset_sizes['train']} training images and {dataset_sizes['val']} validation images.")
    print(f"Classes: {class_names}")
    return dataloaders, dataset_sizes, class_names

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Classification Training Setup')
    parser.add_argument('--data_dir', type=str, default='./classification_data', help='Directory with classification data')
    parser.add_argument('--batch_size', type=int, default=32, help='Batch size for data loaders')
    parser.add_argument('--image_size', type=int, default=224, help='Image size for resizing')
    args = parser.parse_args()

    device = setup_device()
    data_transforms = get_data_transforms(args.image_size)
    dataloaders, dataset_sizes, class_names = get_data_loaders(args.data_dir, data_transforms, args.batch_size)

    # Example of how to access data
    # inputs, labels = next(iter(dataloaders['train']))
    # print(f"Batch input shape: {inputs.shape}")
    # print(f"Batch labels shape: {labels.shape}")

    print("Classification training setup complete.")
"""

# Write the content to the file
with open('train_classification.py', 'w') as f:
    f.write(file_content)
print("Created train_classification.py with initial setup code.")
print("Initial classification training setup components are ready in train_classification.py.")

Created train_classification.py with initial setup code.
Initial classification training setup components are ready in train_classification.py.


**Reasoning**:
The previous steps successfully created the `train_classification.py` file. Now, to verify its functionality and ensure the initial setup components (device config, data transforms, data loading) are correctly integrated, I will execute the script. This will confirm that the file is executable and performs the expected setup operations.



In [ ]:
import os

print("Verifying train_classification.py setup...")
!python train_classification.py --data_dir ./classification_data
print("Verification complete.")